# Setup

In [ ]:
!pip install scipy==1.6.3
!pip install scikit_optimize==0.8.1
!pip install scikit_learn==0.24.2
! git clone https://github.com/zykhoo/predicting_hamiltonian_dynamics.git

     |████████████████████████████████| 27.4 MB 1.5 MB/s 
  Attempting uninstall: scipy
    Found existing installation: scipy 1.4.1
    Uninstalling scipy-1.4.1:
      Successfully uninstalled scipy-1.4.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.
     |████████████████████████████████| 101 kB 3.9 MB/s 
     |████████████████████████████████| 22.3 MB 7.9 MB/s 
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.0.2
    Uninstalling scikit-learn-1.0.2:
      Successfully uninstalled scikit-learn-1.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
yellowbrick 1.4 requires scikit-learn>=1.0.0, but y

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Run all

In [ ]:
"""# Experiment setup"""

import numpy as np

experiment,sys,dim = "NN","squareroot",2

H = lambda x: np.sqrt(x[0]**4 + x[1]**2+1)
f1 = lambda x: (x[1])/np.sqrt(x[0]**4 + x[1]**2 +1) # dH/dy
f2 = lambda x: -2*(x[0]**3)/np.sqrt(x[0]**4 + x[1]**2 +1) # -dH/dx
spacedim = [(0.,2.),(0.,2.)]
h= 0.1
x0, H0 = 0.,1.
initialcon = [64, 128, 256, 512, 1024]
LR = 0.001
diagdist = np.sum(np.square(np.asarray([spacedim[0][0], spacedim[0][1]]), np.asarray([spacedim[1][0], spacedim[1][1]])))
epsilon = 0.1
fvector = lambda x: np.asarray([f1(x), f2(x)])

"""# Test dataset creation"""

from predicting_hamiltonian_dynamics import groundtruth_2dim
from tqdm import tqdm
import time 

evaluation_length_long = 50
evaluation_length_one = 1

xxlong,yylong = np.linspace(spacedim[0][0], spacedim[0][1], 5), np.linspace(spacedim[1][0], spacedim[1][1], 5)
xlong,ylong = np.meshgrid(xxlong,yylong)
xxshort,yyshort = np.linspace(spacedim[0][0], spacedim[0][1], 20), np.linspace(spacedim[1][0], spacedim[1][1], 20)
xshort,yshort = np.meshgrid(xxshort,yyshort)

trajectories_groundtruth_start_long = np.expand_dims(groundtruth_2dim.classicTrajectory(np.asarray([[0.4],[0.]]),f1,f2,h = 0.1,N=evaluation_length_long,n_h = 1),0)
for i in tqdm(np.expand_dims(np.c_[np.ravel(xlong),np.ravel(ylong)],2)):
  results_200 = np.expand_dims(groundtruth_2dim.classicTrajectory(i,f1,f2,h = 0.1,N=evaluation_length_long,n_h = 1),0)
  trajectories_groundtruth_start_long = np.vstack((trajectories_groundtruth_start_long, results_200))
trajectories_groundtruth_start_long.shape # output no., start and final, p and q, full traj

trajectories_groundtruth_start_short = np.expand_dims(groundtruth_2dim.classicTrajectory(np.asarray([[0.4],[0.]]),f1,f2,h = 0.1,N=evaluation_length_one,n_h = 1),0)
for i in tqdm(np.expand_dims(np.c_[np.ravel(xshort),np.ravel(yshort)],2)):
  results_200 = np.expand_dims(groundtruth_2dim.classicTrajectory(i,f1,f2,h = 0.1,N=evaluation_length_one,n_h = 1),0)
  trajectories_groundtruth_start_short = np.vstack((trajectories_groundtruth_start_short, results_200))
trajectories_groundtruth_start_short.shape # output no., start and final, p and q, full traj

within_array = groundtruth_2dim.get_within_array(trajectories_groundtruth_start_long, spacedim)

100%|██████████| 400/400 [00:00<00:00, 1403.79it/s]


In [ ]:
path = '/content/drive/MyDrive/SSI/Baseline v2/synthetic systems (upload)/'

In [ ]:
from predicting_hamiltonian_dynamics.models import NN_2dim
from predicting_hamiltonian_dynamics.models import GP_2dim
from predicting_hamiltonian_dynamics.models import PINN_2dim
from predicting_hamiltonian_dynamics.models import PIGP_2dim


for i in range(20):
  seed = i
  np.random.seed(seed=seed)
  for ini in initialcon: 

    start, final = groundtruth_2dim.CreateTrainingDataTrajClassicIntRandom(1,ini,spacedim,h,f1,f2,seed = seed,n_h = 1)

    delta = start.copy()
    delta[0,:] = f1(start)
    delta[1,:] = f2(start)

    """# NN"""

    from predicting_hamiltonian_dynamics.models import NN_2dim
    import torch

    if torch.cuda.is_available():
      device=torch.device('cuda')
    else:
      device=torch.device('cpu')
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)


    wholemat, evalmat = NN_2dim.data_preprocessing(start, delta, device)

    import torch.optim as optim
    import time 

    net = NN_2dim.Net(dim,16,dim)
    starttime = time.time() 
    net = NN_2dim.train(net, wholemat, evalmat, optimizer=optim.Adam(net.parameters(), lr=LR), batchsize=10, iter=1600, )
    traintime = time.time()-starttime


    from tqdm import tqdm
    from predicting_hamiltonian_dynamics.models import NN_2dim
    from predicting_hamiltonian_dynamics import metrics


    MSE_long, time_long, MSE_long_naive, time_long_naive, MSE_within, time_within, MSE_within_naive, time_within_naive, MSE_onestep, time_onestep, MSE_vectorfield, time_vectorfield, _, _ = NN_2dim.compute_metrics_NN(net, h, diagdist, xshort, yshort, xlong, ylong, evaluation_length_long, within_array, trajectories_groundtruth_start_long, trajectories_groundtruth_start_short, fvector)

    file_object = open(path + 'SquareRoot_random.txt', 'a')
    file_object.write('NN, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s \n' %(ini, seed, traintime, MSE_long, time_long, MSE_long_naive, time_long_naive, MSE_within, time_within, MSE_within_naive, time_within_naive, MSE_onestep, time_onestep, MSE_vectorfield, time_vectorfield))
    file_object.close()


    """# GP"""

    from sklearn.gaussian_process import GaussianProcessRegressor
    from sklearn.gaussian_process.kernels import RBF
    from sklearn import preprocessing

    output = delta
    scaler = preprocessing.MinMaxScaler()
    input = scaler.fit_transform(start.transpose())
    kernel = 1 * RBF(length_scale=2., length_scale_bounds="fixed")
    gaussian_process = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=0, optimizer = None, random_state = seed)
    starttime = time.time() 
    gaussian_process.fit(input, output.transpose())
    traintime = time.time()-starttime
    gaussian_process.predict(scaler.transform([[0.7,0.8]]))

    MSE_long, time_long, MSE_long_naive, time_long_naive, MSE_within, time_within, MSE_within_naive, time_within_naive, MSE_onestep, time_onestep, MSE_vectorfield, time_vectorfield, _, _ = GP_2dim.compute_metrics_GP(gaussian_process, scaler, h, diagdist, xshort, yshort, xlong, ylong, evaluation_length_long, within_array, trajectories_groundtruth_start_long, trajectories_groundtruth_start_short, fvector)

    file_object = open(path + 'SquareRoot_random.txt', 'a')
    file_object.write('GP, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s \n' %(ini, seed, traintime, MSE_long, time_long, MSE_long_naive, time_long_naive, MSE_within, time_within, MSE_within_naive, time_within_naive, MSE_onestep, time_onestep, MSE_vectorfield, time_vectorfield))
    file_object.close()


    """# PINN"""

    from predicting_hamiltonian_dynamics.models import PINN_2dim
    import torch

    if torch.cuda.is_available():
      device=torch.device('cuda')
    else:
      device=torch.device('cpu')
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

    wholemat, evalmat = PINN_2dim.data_preprocessing(start, delta,device)
    net=PINN_2dim.Net(dim,16,1)
    starttime = time.time()
    net,avg_vallosses,avg_lossli,avg_f1li,avg_f2li,avg_f3li,avg_f4li=PINN_2dim.train(net,bs=10,num_epoch=5000,initial_conditions=initialcon,device=device,
                                                                                    wholemat=wholemat,evalmat=evalmat,x0=x0,H0=H0,dim=dim,LR=LR,patience=500,c1=1,c2=1,c3=1,c4=1)
    traintime = time.time()-starttime

    MSE_long, time_long, MSE_long_naive, time_long_naive, MSE_within, time_within, MSE_within_naive, time_within_naive, MSE_onestep, time_onestep, MSE_vectorfield, time_vectorfield, _, _ = PINN_2dim.compute_metrics_PINN(net, device, h, diagdist, xshort, yshort, xlong, ylong, evaluation_length_long, within_array, trajectories_groundtruth_start_long, trajectories_groundtruth_start_short, fvector)
    
    file_object = open(path + 'SquareRoot_random.txt', 'a')
    file_object.write('PINN, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s \n' %(ini, seed, traintime, MSE_long, time_long, MSE_long_naive, time_long_naive, MSE_within, time_within, MSE_within_naive, time_within_naive, MSE_onestep, time_onestep, MSE_vectorfield, time_vectorfield))
    file_object.close()

    """# PIGP"""

    from predicting_hamiltonian_dynamics.models import PIGP_2dim
    GP = PIGP_2dim.BertalanGP()
    starttime = time.time()
    GP.train(start,delta,h)
    traintime = time.time()-starttime

    MSE_long, time_long, MSE_long_naive, time_long_naive, MSE_within, time_within, MSE_within_naive, time_within_naive, MSE_onestep, time_onestep, MSE_vectorfield, time_vectorfield, _, _ = PIGP_2dim.compute_metrics_PIGP(GP, h, diagdist, xshort, yshort, xlong, ylong, evaluation_length_long, within_array, trajectories_groundtruth_start_long, trajectories_groundtruth_start_short, fvector)

    file_object = open(path + 'SquareRoot_random.txt', 'a')
    file_object.write('PIGP, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s \n' %(ini, seed, traintime, MSE_long, time_long, MSE_long_naive, time_long_naive, MSE_within, time_within, MSE_within_naive, time_within_naive, MSE_onestep, time_onestep, MSE_vectorfield, time_vectorfield))
    file_object.close()


Output hidden; open in https://colab.research.google.com to view.